##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Inicio rápido de las API de TensorFlow Core

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/core/quickstart_core"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/core/quickstart_core.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/core/quickstart_core.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver el código fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/core/quickstart_core.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a> </td>
</table>

En este tutorial de inicio rápido se muestra cómo utilizar las API de bajo nivel de [TensorFlow Core](https://www.tensorflow.org/guide/core) para crear y entrenar un modelo de regresión lineal múltiple que prediga la eficiencia del combustible. Para ello, se utiliza el conjunto de datos [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg){::.external} que contiene datos de eficiencia del combustible para automóviles de finales de la década de 1970 y principios de la década de 1980.

Seguirá las etapas típicas de un proceso de aprendizaje automático:

1. Carga el conjunto de datos.
2. Construye una [tubería de entrada.](../data.ipynb).
3. Crea un modelo de [regresión lineal](https://developers.google.com/machine-learning/glossary#linear-regression){:.external} múltiple.
4. Evalúe el rendimiento del modelo.

## Preparación

Importe TensorFlow y otras bibliotecas necesarias para comenzar:

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
print("TensorFlow version:", tf.__version__)
# Set a random seed for reproducible results 
tf.random.set_seed(22)

## Cargar y preprocesar el conjunto de datos

A continuación, deberá cargar y preprocesar el conjunto de datos [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg){:.external} del [Repositorio de aprendizaje automático de la UCI](https://archive.ics.uci.edu/ml/){:.external}. Este conjunto de datos utiliza una serie de características cuantitativas y cualitativas, como cilindros, desplazamiento, caballos de fuerza y peso, para predecir la eficiencia del combustible de los automóviles a finales de la década de 1970 y principios de la década de 1980.

El conjunto de datos contiene algunos valores desconocidos. Asegúrese de eliminar cualquier valor que falte con `pandas.DataFrame.dropna`, y convierta el conjunto de datos a un tipo de tensor `tf.float32` con las funciones `tf.convert_to_tensor` y `tf.cast`.

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

dataset = pd.read_csv(url, names=column_names, na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

dataset = dataset.dropna()
dataset_tf = tf.convert_to_tensor(dataset, dtype=tf.float32)
dataset.tail()

Después, divida el conjunto de datos en conjuntos de entrenamiento y de prueba. Asegúrese de mezclar el conjunto de datos con `tf.random.shuffle` para evitar divisiones sesgadas.

In [ ]:
dataset_shuffled = tf.random.shuffle(dataset_tf, seed=22)
train_data, test_data = dataset_shuffled[100:], dataset_shuffled[:100]
x_train, y_train = train_data[:, 1:], train_data[:, 0]
x_test, y_test = test_data[:, 1:], test_data[:, 0]

Ejecute la ingeniería básica de características mediante la encriptación de una sola vez de la característica `"Origin"`. La función `tf.one_hot` es útil para transformar esta columna de categorías en 3 columnas binarias separadas.

In [ ]:
def onehot_origin(x):
  origin = tf.cast(x[:, -1], tf.int32)
  # Use `origin - 1` to account for 1-indexed feature
  origin_oh = tf.one_hot(origin - 1, 3)
  x_ohe = tf.concat([x[:, :-1], origin_oh], axis = 1)
  return x_ohe

x_train_ohe, x_test_ohe = onehot_origin(x_train), onehot_origin(x_test)
x_train_ohe.numpy()

En este ejemplo se muestra un problema de regresión múltiple con predictores o características en escalas muy diferentes. Por lo tanto, es beneficioso estandarizar los datos para que cada característica tenga un promedio cero y una varianza por unidad. Utilice las funciones `tf.reduce_mean` y `tf.math.reduce_std` para la estandarización. La predicción del modelo de regresión puede entonces desestandarizarse para obtener su valor en términos de las unidades originales.

In [ ]:
class Normalize(tf.Module):
  def __init__(self, x):
    # Inicializa el promedio y la desviación estándar para la normalización
    self.mean = tf.math.reduce_mean(x, axis=0)
    self.std = tf.math.reduce_std(x, axis=0)

  def norm(self, x):
    # Normaliza la entrada
    return (x - self.mean)/self.std

  def unnorm(self, x):
    # Anula la normalización de la entrada
    return (x * self.std) + self.mean

In [ ]:
norm_x = Normalize(x_train_ohe)
norm_y = Normalize(y_train)
x_train_norm, y_train_norm = norm_x.norm(x_train_ohe), norm_y.norm(y_train)
x_test_norm, y_test_norm = norm_x.norm(x_test_ohe), norm_y.norm(y_test)

## Cree un modelo de aprendizaje automático

Cree un modelo de regresión lineal con las API de TensorFlow Core. A continuación encontrará la ecuación para la regresión lineal múltiple:

$${\mathrm{Y}} = {\mathrm{X}}w + b$$

donde:

- $\underset{m\times 1}{\mathrm{Y}}$: vector objetivo
- $\underset{m\times n}{\mathrm{X}}$: matriz de características
- $\underset{n\times 1}w$: vector de ponderación
- $b$: sesgo

Al utilizar el decorador `@tf.function`, se rastrea el código Python correspondiente para generar una gráfica de TensorFlow que se pueda llamar. Este enfoque es beneficioso para guardar y cargar el modelo después del entrenamiento. También puede mejorar el rendimiento de los modelos con muchas capas y operaciones complejas. 

In [ ]:
class LinearRegression(tf.Module):

  def __init__(self):
    self.built = False

  @tf.function
  def __call__(self, x):
    # Inicializa los parámetros del modelo en la primera llamada
    if not self.built:
      # Genera aleatoriamente el vector de ponderación y el término del sesgo
      rand_w = tf.random.uniform(shape=[x.shape[-1], 1])
      rand_b = tf.random.uniform(shape=[])
      self.w = tf.Variable(rand_w)
      self.b = tf.Variable(rand_b)
      self.built = True
    y = tf.add(tf.matmul(x, self.w), self.b)
    return tf.squeeze(y, axis=1)

Para cada ejemplo, el modelo ofrece una predicción de las MPG del automóvil de entrada al calcular la suma ponderada de sus características más un término del sesgo. Esta predicción puede desestandarizarse para obtener su valor en términos de las unidades originales.

In [ ]:
lin_reg = LinearRegression()
prediction = lin_reg(x_train_norm[:1])
prediction_unnorm = norm_y.unnorm(prediction)
prediction_unnorm.numpy()

## Defina una función de pérdida

Ahora, defina una función de pérdida para evaluar el rendimiento del modelo durante el proceso de entrenamiento.

Dado que los problemas de regresión utilizan resultados continuos, el error cuadrático medio (ECM) es una opción ideal para la función de pérdida. El MSE se define mediante la siguiente ecuación:

$$MSE = \frac{1}{m}\sum_{i=1}^{m}(\hat{y}_i -y_i)^2$$

donde:

- $\hat{y}$: vector de predicciones
- $y$: vector de objetivos verdaderos

El objetivo de este problema de regresión es encontrar el vector de ponderación óptimo, $w$, y el sesgo, $b$, que minimice la función de pérdida MSE. 

In [ ]:
def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.square(y_pred - y))

## Entrene y evalúe su modelo

El uso de minilotes para el entrenamiento proporciona eficiencia de memoria y una convergencia más rápida. La API `tf.data.Dataset` tiene funciones útiles para el procesamiento por lotes y la mezcla. La API le permite construir complejas tuberías de entrada a partir de piezas simples y reutilizables. Obtenga más información sobre la construcción de tuberías de entrada de TensorFlow en [esta guía](https://www.tensorflow.org/guide/data).

In [ ]:
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_norm, y_train_norm))
train_dataset = train_dataset.shuffle(buffer_size=x_train.shape[0]).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_norm, y_test_norm))
test_dataset = test_dataset.shuffle(buffer_size=x_test.shape[0]).batch(batch_size)

A continuación, escriba un bucle de entrenamiento para actualizar de forma iterativa los parámetros de su modelo haciendo uso de la función de pérdida MSE y sus gradientes con respecto a los parámetros de entrada.

Este método iterativo se denomina [descenso de gradiente](https://developers.google.com/machine-learning/glossary#gradient-descent){:.external}. En cada iteración, los parámetros del modelo se actualizan al dar un paso en la dirección opuesta de sus gradientes calculados. El tamaño de este paso se determina por la tasa de aprendizaje, la cual es un hiperparámetro que se puede configurar. Recordemos que el gradiente de una función indica la dirección de su ascenso más pronunciado; por lo tanto, dar un paso en la dirección opuesta indica la dirección de descenso más pronunciado, lo que en última instancia ayuda a minimizar la función de pérdida MSE.

In [ ]:
# Establece los parámetros del entrenamiento
epochs = 100
learning_rate = 0.01
train_losses, test_losses = [], []

# Bucle de entrenamiento de formato
for epoch in range(epochs):
  batch_losses_train, batch_losses_test = [], []

  # Itera mediante los datos de entrenamiento
  for x_batch, y_batch in train_dataset:
    with tf.GradientTape() as tape:
      y_pred_batch = lin_reg(x_batch)
      batch_loss = mse_loss(y_pred_batch, y_batch)
    # Actualiza parámetros con respecto a los cálculos de gradiente
    grads = tape.gradient(batch_loss, lin_reg.variables)
    for g,v in zip(grads, lin_reg.variables):
      v.assign_sub(learning_rate * g)
    # Realiza un seguimiento del rendimiento del entrenamiento a nivel de lote 
    batch_losses_train.append(batch_loss)
  
  # Itera mediante los datos de la prueba
  for x_batch, y_batch in test_dataset:
    y_pred_batch = lin_reg(x_batch)
    batch_loss = mse_loss(y_pred_batch, y_batch)
    # Mantiene un registro del rendimiento de las pruebas a nivel de lote 
    batch_losses_test.append(batch_loss)

  # Mantiene un registro del rendimiento del modelo a nivel de época
  train_loss = tf.reduce_mean(batch_losses_train)
  test_loss = tf.reduce_mean(batch_losses_test)
  train_losses.append(train_loss)
  test_losses.append(test_loss)
  if epoch % 10 == 0:
    print(f'Mean squared error for step {epoch}: {train_loss.numpy():0.3f}')

# Pérdidas finales de la salida
print(f"\nFinal train loss: {train_loss:0.3f}")
print(f"Final test loss: {test_loss:0.3f}")

Grafique los cambios en la pérdida de MSE a lo largo del tiempo. Calcule las métricas de rendimiento en un [conjunto de validación](https://developers.google.com/machine-learning/glossary#validation-set){:. external} designado o un [conjunto de prueba](https://developers.google.com/machine-learning/glossary#test-set){:. external} y asegure que el modelo no se desborda en el conjunto de datos de entrenamiento y puede generalizar bien a los datos invisibles.

In [ ]:
matplotlib.rcParams['figure.figsize'] = [9, 6]

plt.plot(range(epochs), train_losses, label = "Training loss")
plt.plot(range(epochs), test_losses, label = "Testing loss")
plt.xlabel("Epoch")
plt.ylabel("Mean squared error loss")
plt.legend()
plt.title("MSE loss vs training iterations");

Parece que el modelo hace un buen trabajo al ajustar los datos de entrenamiento y al mismo tiempo generalizar los datos de prueba invisibles.

## Guardar y cargar el modelo

Comience por crear un módulo de exportación que reciba los datos sin procesar y realice las siguientes operaciones:

- Extracción de características
- Normalización
- Predicción
- Desnormalización

In [ ]:
class ExportModule(tf.Module):
  def __init__(self, model, extract_features, norm_x, norm_y):
    # Inicializa las funciones de pre y postprocesamiento
    self.model = model
    self.extract_features = extract_features
    self.norm_x = norm_x
    self.norm_y = norm_y

  @tf.function(input_signature=[tf.TensorSpec(shape=[None, None], dtype=tf.float32)]) 
  def __call__(self, x):
    # Ejecuta ExportModule para nuevos puntos de datos
    x = self.extract_features(x)
    x = self.norm_x.norm(x)
    y = self.model(x)
    y = self.norm_y.unnorm(y)
    return y 

In [ ]:
lin_reg_export = ExportModule(model=lin_reg,
                              extract_features=onehot_origin,
                              norm_x=norm_x,
                              norm_y=norm_y)

Si desea guardar el modelo en su estado actual, utilice la función `tf.saved_model.save`. Para cargar un modelo guardado para hacer predicciones, utilice la función `tf.saved_model.load`.

In [ ]:
import tempfile
import os

models = tempfile.mkdtemp()
save_path = os.path.join(models, 'lin_reg_export')
tf.saved_model.save(lin_reg_export, save_path)

In [ ]:
lin_reg_loaded = tf.saved_model.load(save_path)
test_preds = lin_reg_loaded(x_test)
test_preds[:10].numpy()

## Conclusión

¡Felicidades! Entrenó un modelo de regresión usando las API de bajo nivel de TensorFlow Core.

Para obtener más ejemplos de uso de las API de TensorFlow Core, consulte las siguientes guías:

- [Regresión logística](./logistic_regression_core.ipynb) para la clasificación binaria
- [Perceptrones multicapa](./mlp_core.ipynb) para el reconocimiento de dígitos escritos a mano
